In [10]:
# Пример скрипта для получения списка моделей
import os
from openai import OpenAI

# Укажи адрес твоего Ollama сервера
# Вместо "YOUR_OLLAMA_BASE_URL" подставь реальный адрес,
# например "http://localhost:11434"
client = OpenAI(
    base_url="http://localhost:11434/v1", # Используй адрес твоего Ollama + /v1
    api_key="ollama", # Фиктивный ключ
)

try:
    response = client.models.list()
    print("Available models:")
    if response.data:
        for model in response.data:
            # Ollama возвращает имя модели (тег) в поле 'id'
            print(f"- {model.id}")
    else:
        print("No models found.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Ensure Ollama server is running.")


Available models:
- qwen2.5:7b
- qwen2.5:latest
- llama3.2:3b
- hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q5_K_S
- hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q4_0
- hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q6_K
- hf.co/ruslandev/llama-3-8b-gpt-4o-ru1.0-gguf:F16
- hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0
- hf.co/ruslandev/llama-3-8b-gpt-4o-ru1.0-gguf:Q4_K_M
- llama-ru:latest
- hf.co/ruslandev/llama-3-8b-gpt-4o-ru1.0-gguf:Q8_0
- phi3:mini


# Base

In [12]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:11434/v1", # Используй адрес твоего Ollama + /v1
    api_key="ollama", # Фиктивный ключ
)

# MODEL_NAME = "qwen2.5:7b"
# MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0"
MODEL_NAME = "hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q4_0"
# MODEL_NAME = "hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q6_K"

In [17]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Why is the sky blue?"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Синий цвет неба объясняется явлением рассеяния света в атмосфере Земли, известным как рассеяние Ми. Атомы кислорода и азота, из которых состоит воздух, сильнее всего рассеивают синий свет, делая его видимым больше голубого цвета.CPU times: user 116 ms, sys: 13.6 ms, total: 129 ms
Wall time: 1.68 s


In [14]:
%%time

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": "Почему небо голубое? Пожалуйста, в 2−3 предложениях."}],
    temperature=1.2,
    max_tokens=1000,            # не более ~100 токенов в ответе
    stream=False,              # полная выдача разом, без «кусков»
)
print(response.choices[0].message.content)

Небо голубое из-за атмосферы Земли. Солнечные лучи проходят сквозь атмосферу и частично рассеиваются благодаря молекулам воздуха, особенно с присутствием азота и кислорода и небольших частиц пыли. Рассеянный свет преимущественно синий или голубой, поэтому мы видим небо голубым.
CPU times: user 11 ms, sys: 4.79 ms, total: 15.8 ms
Wall time: 1.79 s


In [10]:
%%time

# Стримим часть за частью
response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Почему небо голубое? Пожалуйста, ответь в 2−3 предложениях."}],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)

full_text = ""
for chunk in response_stream:
    # Берём контент у delta напрямую, если None — подставляем пустую строку
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)
    full_text += content

print("\n" + "-"*40)
print(full_text)

Небо выглядит голубым из-за процесса рассеяния Рэлея, при котором солнечный свет рассеивается атмосферными частицами воздуха и водяного пара, и поступает в нашу глаза в основном в синей частоте. Однако на самом деле цвет неба может варьироваться в зависимости от времени суток, погоды и даже загрязнения атмосферы.
----------------------------------------
Небо выглядит голубым из-за процесса рассеяния Рэлея, при котором солнечный свет рассеивается атмосферными частицами воздуха и водяного пара, и поступает в нашу глаза в основном в синей частоте. Однако на самом деле цвет неба может варьироваться в зависимости от времени суток, погоды и даже загрязнения атмосферы.
CPU times: user 243 ms, sys: 41.2 ms, total: 285 ms
Wall time: 3.77 s


In [11]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Привет"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Здравствуйте! Как я могу помочь вам сегодня?CPU times: user 55.6 ms, sys: 6.23 ms, total: 61.8 ms
Wall time: 709 ms


In [12]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Привет"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Здравствуйте! Как я могу вам помочь сегодня?CPU times: user 48 ms, sys: 10 ms, total: 58 ms
Wall time: 663 ms


In [13]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Расскажи о принципах программирования"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Прinciples of programming (принципы программирования) включают в себя базовые концепции, которыми следует придерживаться разработчикам для создания эффективных и легких в поддержке программ. Вот несколько ключевых принципов:

1. **Читаемость кода**: Код должен быть легко читабельным и понятным другим разработчикам (и даже самому себе). Используйте ясные именования переменных, функций и классов.

2. **Модульность и агрегация**: Разбейте проект на отдельные части или модули, которые можно использовать и изменять независимо друг от друга. Это упрощает разработку, тестирование и поддержку.

3. **Декомпозиция**: Решите сложную задачу с помощью более простых подзадач. Это помогает в управлении проектами и делает код более понятным.

4. **Совместимость и переносимость**: Ваш код должен работать в различных окружениях и быть совместимым с разными версиями библиотек или стандартов.

5. **Принцип открытости/закрытости (Open/Closed Principle)**: Программные entities (например, классы, модули, фун

In [14]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Привет, как прошел твой день?"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Здравствуйте! К сожалению, у меня нет чувств или личной жизни, поэтому я не могу описать свой день в человеческом понимании этого слова. Но я готов помочь вам с любыми задачами или вопросами, которые у вас могут быть!CPU times: user 158 ms, sys: 21.7 ms, total: 179 ms
Wall time: 2.33 s


In [15]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Привет, какие планы на сегодня?"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Здравствуйте! У меня как искусственного интеллекта нет个人计划，但我随时准备帮助您解答问题或提供所需的信息。您今天有什么安排吗？如果您需要任何建议或帮助，请告诉我！CPU times: user 118 ms, sys: 13.9 ms, total: 132 ms
Wall time: 1.73 s


In [16]:
%%time

response_stream = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": "Ты полезный ассистент. Всегда отвечай на русском языке. Твой ответ должен быть серьезным и локаничным."},
        {"role": "user", "content": "Как менялось предназначение здания музея Прадо на протяжении истории, и какие ключевые события повлияли на его становление?"}
    ],
    temperature=0.8,
    max_tokens=1000,
    stream=True,
)
for chunk in response_stream:
    content = chunk.choices[0].delta.content or ""
    print(content, end="", flush=True)

Здание Музея Прадо в مدريدе Испании имеет богатую историю, которая отражается в изменениях его функций. Давайте рассмотрим ключевые этапы этого процесса.

1. **Построение в качестве дворца**: В 1765 году здание было построено как частный дворец для маркизы де Прадо, супруги короля Испании Фердинанда VI и Изабель II.

2. **Перевод в государственное владение**: После того, как дворец был продан короне в 1808 году из-за долгов маркизы де Прадо, король Карлос IV передал его властям для использования военных целей.

3. **Можайский музей (1819)**: В это время здание было использовано как хранилище коллекции энтрерридансов, солдат, захваченных испанскими войсками в военной кампании против Франции. После этого периода, когда Франция оккупировала Испанию и король Карлос IV отрекся от престола (1808-1814), здание было временно использовано как Можайский музей.

4. **Переход к музею**: После освобождения Испании от французского ига, в 1819 году Испания приняла решение открыть музей в здании Прадо